In [1]:
import trimesh
import numpy as np
from io_utils import stdout_redirected
from dvidutils import encode_faces_to_custom_drc_bytes
import dask
import utils
import time

from collections import namedtuple 
import openmesh as om
import sys
import os
import pyfqmr

id=17
os.system(f"rm -rf test/simpler_multires//{id}*")

Fragment = namedtuple('Fragment', ['draco_bytes', 'position','offset'])

def get_face_indices_in_range(mesh, face_mins, stop):
    max_edge_length = np.max(mesh.edges_unique_length)
    rows = np.where(face_mins[:,0]<stop+max_edge_length)
    return rows[0]

def renumber_vertex_indices(faces, vertex_indices_in_range):
    def renumber_indicies(a, val_old, val_new):
        arr = np.empty(a.max()+1, dtype=val_new.dtype)
        arr[val_old] = val_new
        return arr[a]
    
    faces = np.reshape(faces,-1)
    faces=renumber_indicies(faces,vertex_indices_in_range,np.arange(len(vertex_indices_in_range)))

    return np.reshape(faces,(-1,3))

def my_slice_faces_plane(v,f,plane_normal, plane_origin):
    # Wrapper for trimesh slice_faces_plane, checks that there are vertices and faces and catches an error that happens if the whole mesh is to one side

    if len(v)>0 and len(f)>0:
        try:
            v, f = trimesh.intersections.slice_faces_plane(
                    v, f, plane_normal=plane_normal, plane_origin=plane_origin)
        except ValueError as e:
            if str(e) != "input must be 1D integers!":
                raise
            else:
                pass

    return v,f

def update_dict(combined_fragments_dictionary, fragment_origin, v, f):              
    if fragment_origin in combined_fragments_dictionary:
        [v_combined, f_combined] = combined_fragments_dictionary[fragment_origin]

        f_combined = np.vstack((f_combined, f+len(v_combined)))
        v_combined = np.vstack((v_combined, v))

        combined_fragments_dictionary[fragment_origin] = [v_combined, f_combined]
    else:
        combined_fragments_dictionary[fragment_origin] = [v, f]
    
    return combined_fragments_dictionary

@dask.delayed
def generate_mesh_decomposition(v,f,lod_0_box_size, start_fragment, end_fragment, x_start, x_end, current_lod, starting_lod):
    combined_fragments_dictionary = {}
    fragments = []

    nyz, nxz, nxy = np.eye(3)

    if current_lod != starting_lod:
        sub_box_size = lod_0_box_size*2**(current_lod-1 - starting_lod)
        start_fragment*=2 #since want it to be divisible by 2x2x2 subregions
        end_fragment*=2
        x_start *= 2
        x_end *= 2
    else:
        sub_box_size = lod_0_box_size
    
    for x in range(x_start, x_end):
        vx, fx = my_slice_faces_plane(
            v, f, plane_normal=-nyz, plane_origin=nyz*(x+1)*sub_box_size)

        for y in range(start_fragment[1], end_fragment[1]):
            vy, fy = my_slice_faces_plane(vx, fx, plane_normal=-nxz, plane_origin=nxz*(y+1)*sub_box_size)

            for z in range(start_fragment[2], end_fragment[2]):
                vz, fz = my_slice_faces_plane(
                    vy, fy, plane_normal=-nxy, plane_origin=nxy*(z+1)*sub_box_size)                    

                if current_lod != starting_lod:
                    fragment_origin = tuple(  np.asarray([x,y,z]) // 2)
                else:
                    fragment_origin = tuple(  np.asarray([x,y,z]) )

                
                combined_fragments_dictionary = update_dict(combined_fragments_dictionary, fragment_origin, vz, fz)

                # if len(vz)>0:
                #     if current_lod != starting_lod:
                #         combined_fragments_dictionary = update_dict(combined_fragments_dictionary, fragment_origin, vz, fz)
                #     else:
                #         draco_bytes = encode_faces_to_custom_drc_bytes(vz , np.zeros(np.shape(vz)), fz, np.asarray(3*[lod_0_box_size]), np.asarray(fragment_origin)*lod_0_box_size, position_quantization_bits = 10) 
                #         if len(draco_bytes)>12:
                #             fragment = Fragment(draco_bytes, np.asarray( fragment_origin ), len(draco_bytes))   
                #             fragments.append(fragment)

                vy, fy = my_slice_faces_plane(
                   vy, fy, plane_normal=nxy, plane_origin=nxy*(z+1)*sub_box_size)

            vx, fx = my_slice_faces_plane(
                vx, fx, plane_normal=nxz, plane_origin=nxz*(y+1)*sub_box_size)
        
        v, f = my_slice_faces_plane(
            v, f, plane_normal=nyz, plane_origin=nyz*(x+1)*sub_box_size)

    # if current_lod==starting_lod:
    #     return fragments
    
    #return combined_fragments_dictionary    
    for fragment_origin,[v,f] in combined_fragments_dictionary.items():
        current_box_size = lod_0_box_size*2**(current_lod-starting_lod)
        draco_bytes = encode_faces_to_custom_drc_bytes(
            v , np.zeros(np.shape(v)), f, np.asarray(3*[current_box_size]), np.asarray(fragment_origin)*current_box_size, position_quantization_bits = 10)
       
        if len(draco_bytes)>12:
            fragment = Fragment(draco_bytes, np.asarray( fragment_origin ), len(draco_bytes))   
            fragments.append(fragment)
            
    return fragments

def pyfqmr_decimate(v, f, fraction):
    mesh_simplifier = pyfqmr.Simplify()
    mesh_simplifier.setMesh(v, f)
    mesh_simplifier.simplify_mesh(
            target_count=len(f)//2, aggressiveness=7, preserve_border=False, verbose=0)
    v, f, _ = mesh_simplifier.getMesh()
    return v,f

def decimate(v, f, fraction):
    target = max(4, int(fraction * len(v)))

    try:
        sys.stderr.fileno()
    except:
        # Can't redirect stderr if it has no file descriptor.
        # Just let the output spill to wherever it's going.
        m = om.TriMesh(v, f)
    else:
        # Hide stderr, since OpenMesh construction is super noisy.
        with stdout_redirected(stdout=sys.stderr):
            m = om.TriMesh(v, f)
        
        h = om.TriMeshModQuadricHandle()
        d = om.TriMeshDecimater(m)
        d.add(h)
        d.module(h).unset_max_err()
        d.initialize()

        print(
            f"Attempting to decimate to {target} (Reduce by {len(mesh.vertices) - target})")
        eliminated_count = d.decimate_to(target)
        print(f"Reduced by {eliminated_count}")
        m.garbage_collection()

    v = m.points().astype(np.float32)
    f = m.face_vertex_indices().astype(np.uint32)

    return v,f

In [2]:
mesh = trimesh.load(
        f"/groups/cosem/cosem/ackermand/meshesForWebsite/res1decimation0p1/jrc_hela-1/er_seg/{id}.obj")
print("read")
print(np.min(mesh.vertices, axis=0))
print(np.max(mesh.vertices, axis=0))
print(len(mesh.faces))

read
[3.49597e+03 3.73832e+00 2.67659e+04]
[27938.8   5679.44 59501.7 ]
15827909


In [3]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=4,
                n_workers=16)


/groups/scicompsoft/home/ackermand/miniconda3/envs/simpler_multires/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43201 instead
  f"Port {expected} is already in use.\n"
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-9a1mm0ly', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-3jz5l5ch', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-vimkgozb', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/multiresolutionMeshes/dask-worker-space/worker-

In [4]:
import dask

nyz, nxz, nxy = np.eye(3)
num_workers = 16
num_lods = 6

lods = list(range(num_lods))


v_whole = mesh.vertices.astype(np.float32)
f_whole = mesh.faces.astype(np.uint32)

mesh = []

lod_0_box_size = 64*4

for current_lod in lods:
    print("calculating frags")
    current_box_size = lod_0_box_size * 2**(current_lod-lods[0])
    start_fragment =  np.maximum( np.min(v_whole, axis=0).astype(int) // current_box_size - 1 , np.array([0,0,0]) )
    end_fragment = np.max(v_whole, axis=0).astype(int) // current_box_size + 1 
    x_stride = int(np.ceil(1.0*(end_fragment[0]-start_fragment[0])/num_workers))
    print("calculated frags")

    results = []

    v = v_whole
    f = f_whole
    print(current_box_size, start_fragment, end_fragment, x_stride)
    if len(lods)>1: #decimate here so don't have to keep original v_whole, f_whole around
        v_whole,f_whole = pyfqmr_decimate(v_whole, f_whole, 4)
        print("decimateeeeee")
    for x in range(start_fragment[0], end_fragment[0]+1,x_stride):
        vx, fx = my_slice_faces_plane(v, f, plane_normal=-nyz, plane_origin=nyz*(x+x_stride)*current_box_size)
        if len(vx)>0:
            results.append( generate_mesh_decomposition(vx, fx, lod_0_box_size, start_fragment, end_fragment, x, x+x_stride, current_lod, lods[0]) )
            v, f = my_slice_faces_plane(v, f, plane_normal=nyz, plane_origin=nyz*(x+x_stride)*current_box_size)

    dask_results = dask.compute(* results)
    fragments = [fragment for fragments in dask_results for fragment in fragments]
    print(len(fragments))
   
    utils.write_files("test/simpler_multires", f"{id}", fragments, current_lod, lods, np.asarray([lod_0_box_size,lod_0_box_size,lod_0_box_size]))
    print(current_lod)


calculating frags
calculated frags
256 [ 12   0 103] [110  23 233] 7
simplified mesh in 55.0127 seconds from 15827909 to 7913953 triangles


tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x7fa1ad5a6fd0>>, <Task finished coro=<Worker.heartbeat() done, defined at /groups/scicompsoft/home/ackermand/miniconda3/envs/simpler_multires/lib/python3.7/site-packages/distributed/worker.py:967> exception=OSError('Timed out during handshake while connecting to tcp://127.0.0.1:34701 after 30 s')>)
Traceback (most recent call last):
  File "/groups/scicompsoft/home/ackermand/miniconda3/envs/simpler_multires/lib/python3.7/site-packages/distributed/comm/core.py", line 321, in connect
    handshake = await asyncio.wait_for(comm.read(), time_left())
  File "/groups/scicompsoft/home/ackermand/miniconda3/envs/simpler_multires/lib/python3.7/asyncio/tasks.py", line 449, in wait_for
    raise futures.TimeoutError()
concurrent.futures._base.TimeoutError

The above exception was the direct cause of the following exception:

Traceback

decimateeeeee


/groups/scicompsoft/home/ackermand/miniconda3/envs/simpler_multires/lib/python3.7/site-packages/distributed/worker.py:3801: UserWarning: Large object of size 29.95 MiB detected in task graph: 
  (array([[24473.59960938,   529.38098145, 56539.    ... , 89, 96, 0, 0)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


60884
0
calculating frags
calculated frags
512 [ 5  0 51] [ 55  12 117] 4
simplified mesh in 27.3232 seconds from 7913953 to 3956976 triangles
decimateeeeee
10145


...

[136944  10162]
1
calculating frags
calculated frags
1024 [ 2  0 25] [28  6 59] 2
simplified mesh in 14.9023 seconds from 3956976 to 1978488 triangles
decimateeeeee
1780


distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 21% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 22% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

[136944  22473   3239]
2
calculating frags
calculated frags
2048 [ 0  0 12] [14  3 30] 1
simplified mesh in 7.6914 seconds from 1978488 to 989243 triangles
decimateeeeee
325


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


[136944  22473   4067    554]
3
calculating frags
calculated frags
4096 [0 0 5] [ 7  2 15] 1
simplified mesh in 3.8219 seconds from 989243 to 494620 triangles
decimateeeeee
68
[136944  22473   4067    873    125]
4
calculating frags
calculated frags
8192 [0 0 2] [4 1 8] 1
simplified mesh in 1.904 seconds from 494620 to 247310 triangles
decimateeeeee
19
[136944  22473   4067    873    222     28]
5


In [5]:
list(range(start_fragment[0], end_fragment[0]+1,x_stride))


[0, 1, 2, 3, 4]

In [6]:
for fragment in fragments:
    if fragment.position[0]==0 and fragment.position[1]==0 and fragment.position[2]==0:
        print("yoyoy")

nyz, nxz, nxy = np.eye(3)
num_workers = 16
num_lods = 6

lods = list(range(num_lods))


v_whole = mesh.vertices.astype(np.float32)
f_whole = mesh.faces.astype(np.uint32)

min_coords = np.min(v_whole, axis=0)
vertex_offsets = np.array([0,0,0])#min_coords
v_whole-=min_coords

lod_0_box_size = 64*4


#start_fragment =  np.min(v_whole, axis=0).astype(int) // lod_0_box_size - 1 
#end_fragment = np.max(v_whole, axis=0).astype(int) // lod_0_box_size + 1 

start_fragment = [0,0,0]#np.maximum(( np.min(v_whole, axis=0).astype(int) // (lod_0_box_size*2**max(lods)) - 1 ) * (lod_0_box_size*2**max(lods)),np.array([0,0,0]))
end_fragment =  ( np.max(v_whole, axis=0).astype(int) // (lod_0_box_size*2**max(lods)) + 1 ) * (lod_0_box_size*2**max(lods))

x_stride = int(np.ceil(1.0*(end_fragment[0]-start_fragment[0])/num_workers))

AttributeError: 'list' object has no attribute 'vertices'

In [7]:
print(start_fragment, end_fragment, x_stride, np.array([vertex_offsets for _ in range(num_lods)]))

[0, 0, 0] [8192 8192 8192] 512 [[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]


In [6]:
print("calculating frags")
current_box_size = lod_0_box_size * 2**(current_lod-lods[0])
start_fragment =  np.maximum( np.min(v_whole, axis=0).astype(int) // current_box_size - 1 , np.array([0,0,0]) )
end_fragment = np.max(v_whole, axis=0).astype(int) // current_box_size + 1 
x_stride = int(np.ceil(1.0*(end_fragment[0]-start_fragment[0])/num_workers))
print("calculated frags")
len(v_whole)

calculating frags
calculated frags


7757083

In [9]:
 # combined_fragments_dictionaries = [fragment for fragments in dask_results for fragment in fragments]
    
    # final_dict = {}


    # for combined_fragments_dictionary in combined_fragments_dictionaries:
    #     for fragment_origin,[v,f] in combined_fragments_dictionary.items():
    #         current_box_size = lod_0_box_size*2**(current_lod-lods[0])
    #         draco_bytes = encode_faces_to_custom_drc_bytes(
    #             v , np.zeros(np.shape(v)), f, np.asarray(3*[current_box_size]), np.asarray(fragment_origin)*current_box_size, position_quantization_bits = 10)
        
    #         if len(draco_bytes)>12:
    #             fragment = Fragment(draco_bytes, np.asarray( fragment_origin ), len(draco_bytes))   
    #             fragments.append(fragment)